<a href="https://colab.research.google.com/github/hughjafro/DS1-Unit-2-Sprint-5---Project-Week/blob/master/Sub_5_CTolbert_DS1_Unit_2_Project_Week.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Overview

In my first Kaggle challenge, I was able to work on data from Taarifa and the Tanzanian Ministry of Water. The goal is to predict which pumps are **functional, which needs repair and which pumps don't work at all**. The predictions are based on a number of variables, such as who manages the pump, when it was installed, etc.

When I create a Python notebook, I tend to do **overkill** in the beginning when installing packages. This is mostly because I hate trying to execute a line of code and realize I didn't have the package installed yet. So, you will see that I installed the most probable packages that I might use.

In [0]:
# Import all packages

import numpy as np
np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
import matplotlib.pyplot as plt
import seaborn as sns

# Encoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing

# For scaling
from sklearn.preprocessing import scale, StandardScaler, Imputer

# Regression models
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate, cross_val_predict, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier


# For ridge regression
from sklearn.linear_model import Ridge

##Load Data
Next I loaded the dataset from my Github account and read to 3 different dataframes 1) training features 2) training labels 3) test features

In [0]:
url_testf = 'https://raw.githubusercontent.com/hughjafro/DS1-Unit-2-Sprint-5---Project-Week/master/test_features.csv'
url_trainf = 'https://raw.githubusercontent.com/hughjafro/DS1-Unit-2-Sprint-5---Project-Week/master/train_features.csv'
url_trainl = 'https://raw.githubusercontent.com/hughjafro/DS1-Unit-2-Sprint-5---Project-Week/master/train_labels.csv'
df_trainf = pd.read_csv(url_trainf)
df_labels = pd.read_csv(url_trainl)
df_test = pd.read_csv(url_testf)

##Check the initial datasets

After the data is loaded, I will always start by looking at basic characteristics of the datasets like shape and first 5 rows.

In [40]:
print(df_trainf.shape)
df_trainf.head()

(59400, 40)


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [41]:
# This will be our target - status_group
print(df_labels.shape)
df_labels.head()

(59400, 2)


,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [42]:
print(df_test.shape)
df_test.head()

(14358, 40)


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


##ID the target

The goal of the competition is to predict the functional nature of a pump, whether it fully works, works but need repair or doesn't work at all.

To avoid future categorical issues, the target feature (status_group) was replace with numerical inputs by replacing functional with 0, functional needs repair with 1 and non functional with 2.

It is important to note for future consideration, that with 3 categories the target feature is no longer a binary classifier. 

***This is more a note to my future self because I spent way too much time wrangling errors that was caused because I treated the target feature as a binary input.***

In [0]:
#Define target feature
target1 = df_labels.status_group.replace({'functional':0,
                                          'functional needs repair':1,
                                          'non functional':2})

In [44]:
# Check target values
target1.value_counts()

0    32259
2    22824
1     4317
Name: status_group, dtype: int64

####Baseline Accuracy

I did a manual calculation of target feature value counts, followed by a normalizing the features, straightway.

In [45]:
# Percent funtional
print('Percent funtional (baseline accuracy): ', 32259 / (32259+22824+4317))

# Percent non-functional
print('\nPercent non-functional: ', 22824 / (32259+22824+4317))

# Percent need repair
print('\nPercent need repair: ', 4317 / (32259+22824+4317))

Percent funtional (baseline accuracy):  0.543080808080808

Percent non-functional:  0.3842424242424242

Percent need repair:  0.07267676767676767


In [46]:
# Check to make sure this is the right method
target1.value_counts(normalize=True)

0    0.543081
2    0.384242
1    0.072677
Name: status_group, dtype: float64

####Majority Class

In [0]:
majority_class = df_labels['status_group'].mode()[0]

y_pred = np.full(shape=df_labels['status_group'].shape, fill_value=majority_class)

In [48]:
accuracy_score(df_labels['status_group'], y_pred)

0.543080808080808

###Initial Classification Report

In [49]:
print(classification_report(df_labels['status_group'], y_pred) )

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                         precision    recall  f1-score   support

             functional       0.54      1.00      0.70     32259
functional needs repair       0.00      0.00      0.00      4317
         non functional       0.00      0.00      0.00     22824

              micro avg       0.54      0.54      0.54     59400
              macro avg       0.18      0.33      0.23     59400
           weighted avg       0.29      0.54      0.38     59400



###Combine test/train dataframes into one dataframe so they can be cleaned at one time

In [50]:
# Remember to use concat to add to the bottome
df_all = pd.concat([df_trainf, df_test])
print(df_all.shape)
df_all.head()

(73758, 40)


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [51]:
# Check NaN's
df_all.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    4418
gps_height                   0
installer                 4443
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 465
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            4119
recorded_by                  0
scheme_management         4816
scheme_name              35005
permit                    3719
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [0]:
# Drop feature with very large missing values  - Scheme name
df_all=df_all.drop(columns='scheme_name')

In [53]:
df_all.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'permit', 'construction_year', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group'],
      dtype='object')

###Fill missing values with the mode of that column

This is easier for categorical columns

In [0]:
df_all['funder'] = df_all['funder'].fillna(df_all['funder'].mode()[0])

In [0]:


cols = ['installer', 'subvillage', 'public_meeting',
        'scheme_management', 'permit']

df_all[cols] = df_all[cols].fillna(df_all.mode().iloc[0])

In [56]:
# Check for nans
df_all.isna().sum()

id                       0
amount_tsh               0
date_recorded            0
funder                   0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
subvillage               0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
w

In [57]:
target1.shape

(59400,)

###Dummy encode data

In [0]:
# Dummy encoder function
def dummyEncode(df):
        columnsToEncode = list(df.select_dtypes(include=['category','object']))
        le = LabelEncoder()
        for feature in columnsToEncode:
            try:
                df[feature] = le.fit_transform(df[feature])
            except:
                print('Error encoding '+feature)
        return df

#Separate into train and test again

In [0]:
# Target is still target1
X = df_all[:-14358] # drop last 14358 rows
test = df_all[-14358:] # keep the last 14358 rows
y = target1

In [60]:
test.shape

(14358, 39)

In [61]:
print(X.shape)
X.head()

(59400, 39)


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,True,GeoData Consultants Ltd,Other,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,VWC,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [62]:
df_ce = dummyEncode(X)
df_ce.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,47,1369,1390,1518,34.938093,-9.856322,37399,0,1,11807,3,11,5,51,1426,109,True,0,7,False,1999,3,1,0,7,4,2,0,6,2,1,1,8,6,0,1,1
1,8776,0.0,309,469,1399,545,34.698766,-2.147466,37195,0,4,15838,9,20,2,103,1576,280,True,0,2,True,2010,3,1,0,11,4,0,2,6,2,2,2,5,3,1,1,1
2,34310,25.0,300,825,686,2048,37.460664,-3.821329,14572,0,5,9074,8,21,4,108,1624,250,True,0,7,True,2009,3,1,0,7,4,4,5,6,2,1,1,0,1,1,2,1
3,67743,0.0,272,1741,263,1852,38.486161,-11.155298,37285,0,7,8982,12,90,63,87,1571,58,True,0,7,True,1986,14,10,5,7,4,0,2,6,2,0,0,3,0,0,2,1
4,19728,0.0,104,20,0,119,31.130847,-1.825359,35529,0,4,7698,4,18,1,26,1687,0,True,0,7,True,0,3,1,0,1,1,0,2,6,2,3,3,5,3,1,1,1


In [63]:
df_ce.shape

(59400, 39)

In [64]:
test_ce = dummyEncode(test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [65]:
test_ce.shape

(14358, 39)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df_ce, y, test_size=0.25, random_state=42, shuffle=True)

In [67]:
# Logistic Regression again
log_reg = LogisticRegression(solver='newton-cg', multi_class='multinomial')
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [68]:
accuracy_score(y_test, y_pred)

0.6411447811447811

In [69]:
y_test.shape, y_pred.shape, X_test.shape

((14850,), (14850,), (14850, 39))

###Make Pipeline

In [70]:
pipeline = make_pipeline(StandardScaler(),
                        LogisticRegression(solver='newton-cg',
                                           multi_class='multinomial'))

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/usr/local/lib/python3.6/dist-packages/sklearn/pipeline.py:331: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [71]:
accuracy_score(y_test, y_pred)

0.6405387205387205

In [72]:
X_test.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
2980,37098,0.0,194,1407,0,390,31.985658,-3.596360,5344,0,3,521,17,17,5,9,453,0,True,0,9,True,0,9,6,3,11,4,6,6,6,2,0,0,7,5,0,6,5
5246,14530,0.0,219,485,0,622,32.832815,-4.944937,28341,0,3,8791,19,14,6,115,2052,0,True,0,7,True,0,4,2,1,7,4,0,2,3,3,2,2,7,5,0,4,3
22659,62607,10.0,300,1529,1675,390,35.488289,-4.242048,13832,0,0,16810,8,21,1,2,213,148,True,0,10,True,2008,3,1,0,9,4,4,5,6,2,2,2,8,6,0,1,1
39888,46053,0.0,135,727,0,807,33.140828,-9.059386,4273,0,2,1502,10,12,6,62,455,0,False,0,7,False,0,8,5,1,7,4,0,2,6,2,3,3,7,5,0,4,3
13361,47083,50.0,283,1842,1109,1567,34.217077,-4.430529,29036,0,0,11362,18,13,1,23,1403,235,True,0,8,True,2011,7,4,2,10,4,4,5,6,2,1,1,3,0,0,2,1


In [73]:
test_ce.shape

(14358, 39)

In [74]:
test_pred = log_reg.predict(test_ce)
print(test_pred)

[2 0 2 2 0 0 2 2 0 0 0 2 2 0 2 2 0 2 2 0 2 2 0 0 2 2 0 0 0 2 0 0 0 0 0 0 0
 0 2 2 2 2 0 2 2 0 0 0 0 0 0 0 2 0 0 0 0 2 0 2 0 0 2 0 0 0 2 0 0 2 0 0 0 0
 0 2 0 0 0 0 0 0 0 0 2 0 0 0 2 0 0 2 0 0 0 0 0 2 0 2 0 0 0 0 2 2 0 0 0 2 0
 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 2 0 0 0 2 2 0 0 0 0 0 0 2 0 0 2 0 0 0 2 0 0
 2 2 2 2 2 0 2 2 0 0 0 0 0 0 2 2 2 0 0 0 0 0 0 2 0 0 2 0 0 0 2 2 2 0 0 2 0
 0 2 0 2 2 0 0 0 0 0 2 2 2 0 0 0 2 0 0 2 0 0 2 0 2 0 0 0 0 0 2 0 0 2 2 0 0
 0 0 0 0 0 2 0 0 0 2 0 0 0 0 0 1 2 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 2 0 2 0 0
 2 2 0 2 0 0 0 2 0 0 2 2 0 0 0 2 0 2 0 2 2 2 0 0 2 2 2 0 0 2 0 2 0 0 0 2 0
 0 0 2 0 0 0 0 0 0 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 2 0 2 0 2 0 0 0 2 0 0 2 2
 0 0 0 0 0 0 0 0 0 0 2 0 0 2 0 2 0 0 0 0 2 2 0 2 0 2 0 2 0 0 2 0 0 2 0 0 0
 2 0 2 0 0 0 2 2 0 2 2 2 0 0 0 2 2 2 0 0 0 0 2 0 2 0 0 0 0 0 0 0 2 2 0 0 2
 0 0 2 2 0 2 0 2 0 0 0 2 0 2 0 2 0 2 2 0 2 2 0 2 0 2 2 0 0 2 2 0 0 2 2 2 0
 0 0 0 2 0 2 0 2 0 0 0 2 0 2 0 0 2 2 0 2 2 0 0 0 2 2 0 2 0 0 0 2 2 2 2 2 0
 0 2 0 0 0 2 2 0 2 2 2 0 

In [0]:
# accuracy_score(y_test, test_pred)

In [0]:
sub4 = pd.DataFrame({'id':test_ce.id,'status_group':test_pred})

In [78]:
sub4.head()

,id,status_group
0,50785,2
1,51630,0
2,17168,2
3,45559,2
4,49871,0


###XGBoost

Using XGBoost was something that I have wanted to do for a while but did not have the opportunity to use...until now. Early in this competition, Sammy Lee asked if it would be fitting to use XGBoost or not. The response was to use whatever would get a qualifying score. So, after getting a qualifying score of greater than 60, I thought I might explore XGBoost for a training and test accuracy score.****

In [0]:
# !pip install xgboost
import xgboost as xgb

Loading data using XGBoost is fairly similar to other methods.

When introducing the train and test sets, a DMatrix is used. A Dmatrix is basically an internal data structure which is optimized for both memory efficiency and training speed.

Then the dmatrix is saved in a binary buffer that helps with speeding up the processing.

I initially had a small setback when specifying the parameters because I forgot that target was not a binary classification but multiclass. The parameter was originally set as binary:logistic, which obviously would not work. So, I had to move to using multi:softmax: to do multiclass classification using the softmax objective and then designate the number of casses.

In [0]:
# load file from already run tests
label_train = y_train
label_test = y_test

dtrain = xgb.DMatrix(X_train, label=label_train)
dtest = xgb.DMatrix(X_test, label=label_test)

# save dmatrix into binary buffer
dtrain.save_binary('dtrain.buffer')
dtest.save_binary('dtrain.buffer')

# specify parameters via map
# param = {
#     'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic'
# } - Doesn't work because my target is not binary - 0,1,2

param = {
    'num_class':3, 'eta':1, 'silent':1, 'objective':'multi:softmax'
}

Next a **watchlist** is defined with a list from our xgboost dmatrix run. The watchlist allows us to monitor the evaluation result on all data in the list.

The resulting data output of the `bst` variable is something that xgboost calls merror. This is a multiclass classification error rate that is calculated as # of wrong cases / # of all cases.

In [81]:
# specify validations set to watch performance
watchlist = [(dtest, 'test'), (dtrain, 'train')]
num_round = 10

print('merror shows wrong cases / all cases')
bst = xgb.train(param, dtrain, num_round, watchlist)


merror shows wrong cases / all cases
[0]	test-merror:0.276835	train-merror:0.273692
[1]	test-merror:0.267205	train-merror:0.26018
[2]	test-merror:0.25266	train-merror:0.24092
[3]	test-merror:0.244108	train-merror:0.232121
[4]	test-merror:0.239259	train-merror:0.224714
[5]	test-merror:0.233333	train-merror:0.214658
[6]	test-merror:0.232458	train-merror:0.210258
[7]	test-merror:0.227811	train-merror:0.202222
[8]	test-merror:0.224377	train-merror:0.198025
[9]	test-merror:0.222896	train-merror:0.194456


In [82]:
# this is prediction
xg_ypred1 = bst.predict(dtrain)
xg_ypred2 = bst.predict(dtest)

print('Train accuracy score:', accuracy_score(y_train, xg_ypred1))
print('Test accuracy score:', accuracy_score(y_test, xg_ypred2))


# print('error=%f' % (sum(1 for i in range(len(xg_ypred1)) if int(xg_ypred1[i] > 0.5) != labels[i]) / float(len(preds))))
# bst.save_model('0001.model')

Train accuracy score: 0.8055443322109989
Test accuracy score: 0.7771043771043771


##OLD CODE SNIPPETS

###Grid Search

In [0]:
# # Polynomial Regression
# def PolynomialRegression(degree=2, **kwargs):
#     return make_pipeline(PolynomialFeatures(degree),
#                          LinearRegression(**kwargs))

In [0]:
# y = target
# X = df_num

# X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [0]:
# y_train.head()

In [0]:
# param_grid = {
#     'polynomialfeatures__degree': [0,1,2,3]
# }

# gridsearch = GridSearchCV(PolynomialRegression(), param_grid=param_grid,
#                           scoring='neg_mean_absolute_error', cv=3,
#                           return_train_score=True, verbose=10)
# gridsearch.fit(X_train, y_train)

In [0]:
# pd.DataFrame(gridsearch.cv_results_).sort_values(by='rank_test_score')

###Random Forest

In [0]:
# model = RandomForestRegressor(n_estimators=100, max_depth=20)

# scores = cross_validate(model, X_train, y_train,
#                         scoring='neg_mean_absolute_error',
#                         cv=3, return_train_score=True,
#                        return_estimator=True)

# pd.DataFrame(scores)